# BERT

In [1]:
import os

# Force single-GPU mode in Jupyter
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["WORLD_SIZE"] = "1"

In [2]:
import ast
import numpy as np
import pandas as pd
from scipy.sparse import hstack

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    accuracy_score,
    precision_recall_fscore_support,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    log_loss
)

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")

import torch
from torch.utils.data import Dataset

from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding,
    set_seed,
)

from evaluation import *
from data_utils import *

/home/jyao/miniconda3/envs/chatbot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-11-26 00:00:19.972989: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-11-26 00:00:19.973064: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-11-26 00:00:19.974190: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-11-26 00:00:19.981688: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow

In [3]:
# Load and prepare data
train_df, test_df, y, class_names = load_and_prepare_data()

# Get train/val/test splits for BERT
X_train_bert, X_val_bert, X_test_bert, y_train_bert, y_val_bert = prepare_text_pipeline(
    train_df, test_df, y
)

# Ensure positional indexing
y_train_bert = np.asarray(y_train_bert, dtype=int)
y_val_bert   = np.asarray(y_val_bert, dtype=int)

print("Example text:\n", X_train_bert[0][:250])
print("Label:", y_train_bert[0])
print("Classes:", class_names)

Example text:
 ["What is a foreign exchange crisis? What are some notable examples?"] [SEP] ["A foreign exchange crisis refers to a situation where a country faces severe shortage of foreign currencies, usually dollars or euros, to pay for imports or service extern
Label: 1
Classes: {0: 'winner_model_a', 1: 'winner_model_b', 2: 'winner_tie'}


In [4]:
# Torch Dataset
class TextClsDataset(Dataset):
    def __init__(self, texts, labels=None):
        self.texts  = list(texts)
        self.labels = None if labels is None else np.asarray(labels, dtype=int)
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        item = {"text": self.texts[idx]}
        if self.labels is not None:
            item["label"] = int(self.labels[idx])
        return item

ds_train = TextClsDataset(X_train_bert, y_train_bert)
ds_val   = TextClsDataset(X_val_bert, y_val_bert)
ds_test  = TextClsDataset(X_test_bert, labels=None)

In [5]:
from transformers import DebertaV2Tokenizer
# Tokenizer & collator
# MODEL_NAME = "bert-base-uncased"
# MODEL_NAME = "FacebookAI/roberta-base"
MODEL_NAME = "microsoft/deberta-v3-base"
# MODEL_NAME = "microsoft/deberta-v3-large"
max_len = 512
tokenizer = DebertaV2Tokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(
        batch["text"] if isinstance(batch, dict) else batch["text"],
        truncation=True,
        max_length=max_len,
        padding=False,  # dynamic padding via collator
    )

collator = DataCollatorWithPadding(tokenizer=tokenizer)

# HF Trainer expects a dict-like dataset; wrap via map-style adapter
class HFAdapter(Dataset):
    def __init__(self, base_ds, with_labels=True):
        self.base_ds = base_ds
        self.with_labels = with_labels
    def __len__(self): return len(self.base_ds)
    def __getitem__(self, idx):
        ex = self.base_ds[idx]
        toks = tokenizer(
            ex["text"],
            truncation=True,
            max_length=max_len,
            padding=False
        )
        if self.with_labels:
            toks["labels"] = int(ex["label"])
        return toks

hf_train = HFAdapter(ds_train, with_labels=True)
hf_val   = HFAdapter(ds_val,   with_labels=True)
hf_test  = HFAdapter(ds_test,  with_labels=False)

In [6]:
# Model
id2label = {0: class_names[0], 1: class_names[1], 2: class_names[2]}
label2id = {v: k for k, v in id2label.items()}
num_labels = 3

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

model_bert = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id,
    problem_type="single_label_classification"
).to(device)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# TrainingArguments & Trainer
import os, torch, inspect

torch.backends.cuda.matmul.allow_tf32 = True
os.environ.setdefault("TOKENIZERS_PARALLELISM", "false")

# Helper to build TrainingArguments with version-safe keys
def make_training_args(**base_kwargs):
    sig = inspect.signature(TrainingArguments.__init__)
    params = set(sig.parameters.keys())

    # map possibly-renamed keys
    if "evaluation_strategy" in base_kwargs and "evaluation_strategy" not in params and "eval_strategy" in params:
        base_kwargs["eval_strategy"] = base_kwargs.pop("evaluation_strategy")
    if "logging_strategy" in base_kwargs and "logging_strategy" not in params and "log_strategy" in params:
        base_kwargs["log_strategy"] = base_kwargs.pop("logging_strategy")

    # drop keys not supported by this version (e.g., tf32 on older versions)
    safe_kwargs = {k: v for k, v in base_kwargs.items() if k in params}
    return TrainingArguments(**safe_kwargs)

train_args = make_training_args(
    output_dir="runs/bert_cls",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_ratio=0.06,

    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,          # keeps best = lowest eval_loss
    save_total_limit=2,

    bf16=True,
    tf32=True,
    dataloader_num_workers=4,
    log_level="error",
    logging_steps=50,
    report_to="none"
)

trainer = Trainer(
    model=model_bert,
    args=train_args,
    train_dataset=hf_train,
    eval_dataset=hf_val,
    tokenizer=tokenizer,
    data_collator=collator,
)

trainer.train()

/tmp/ipykernel_3641094/3642117575.py:47: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/home/jyao/miniconda3/envs/chatbot/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.091600,1.085849
2,1.059800,1.077582
3,1.009700,1.108454


/home/jyao/miniconda3/envs/chatbot/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/home/jyao/miniconda3/envs/chatbot/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


TrainOutput(global_step=2157, training_loss=1.0604180538051926, metrics={'train_runtime': 2031.7336, 'train_samples_per_second': 67.894, 'train_steps_per_second': 1.062, 'total_flos': 3.629530507838976e+16, 'train_loss': 1.0604180538051926, 'epoch': 3.0})

In [8]:
# Predictions (BERT)
with torch.no_grad():
    pred_logits = trainer.predict(hf_val).predictions
y_proba_val_bert = torch.softmax(torch.tensor(pred_logits), dim=1).cpu().numpy()
y_pred_val_bert  = y_proba_val_bert.argmax(axis=1)

# Test predictions + submission
with torch.no_grad():
    test_logits = trainer.predict(hf_test).predictions
y_proba_test_bert = torch.softmax(torch.tensor(test_logits), dim=1).cpu().numpy()
y_pred_test_bert  = y_proba_test_bert.argmax(axis=1)

/home/jyao/miniconda3/envs/chatbot/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [9]:
print("\n================ BERT EVALUATION ================\n")
# Metrics
_ = eval_metrics(y_val_bert, y_pred_val_bert)
eval_classification_report(y_val_bert, y_pred_val_bert, class_names)
# ROC-AUC
_ = eval_roc_auc(y_val_bert, y_proba_val_bert)
# Log-loss
_ = eval_log_loss(y_val_bert, y_proba_val_bert)
_ = eval_log_loss_per_class(y_val_bert, y_proba_val_bert)


================ BERT EVALUATION ================

*** GLOBAL METRICS ***
Accuracy (Global)      : 0.4147
Precision (Macro Avg)  : 0.4258
Recall (Macro Avg)     : 0.4108
F1-Score (Macro Avg)   : 0.4029

*** PER-CLASS EVALUATION ***
Class                Precision    Recall  F1-Score   Support
------------------------------------------------------------
winner_model_a            0.39      0.61      0.48      4013
winner_model_b            0.42      0.29      0.34      3931
winner_tie                0.47      0.34      0.39      3552
------------------------------------------------------------
Macro Avg                 0.43      0.41      0.40     34488
Weighted Avg              0.42      0.41      0.40     34488

*** ROC-AUC EVALUATION ***
ROC-AUC (OvR) : 0.5895

*** LOG-LOSS EVALUATION ***
Log-loss      : 1.0776

*** LOG-LOSS PER CLASS ***
Class 0: 0.9487  (n=4013)
Class 1: 1.0534  (n=3931)
Class 2: 1.2499  (n=3552)


In [10]:
# Confusion Matrix + Plot
cm_bert = eval_confusion_matrix(y_val_bert, y_pred_val_bert, n_classes=y_proba_val_bert.shape[1])
plot_confusion_matrix(cm_bert, class_names, title="Confusion Matrix — DEBERT", save_path="results/confusion_matrix/confusion_matrix_debert.png")


Confusion Matrix (rows=true, cols=pred):
 [[2442  871  700]
 [2151 1130  650]
 [1642  715 1195]]
Saved plot to: images/confusion_matrix/confusion_matrix_debert.png


In [11]:
# ROC Curves
plot_roc_curves(y_val_bert, y_proba_val_bert, class_names, title_prefix="DEBERT ROC", save_path="results/roc/roc_debert.png")

Saved plot to: images/roc/roc_debert.png


In [12]:
save_roc_to_csv(y_val_bert, y_proba_val_bert, "DEBERT", fold_idx=1)

Saved ROC data for class 0 (AUC=0.5723) → results/roc/DEBERT_fold1_class0.csv
Saved ROC data for class 1 (AUC=0.5786) → results/roc/DEBERT_fold1_class1.csv
Saved ROC data for class 2 (AUC=0.6177) → results/roc/DEBERT_fold1_class2.csv


In [13]:
submission_bert = build_submission(
    test_df=test_df,
    y_pred_test=y_pred_test_bert,
    y_proba_test=y_proba_test_bert,
    model_name="debert"
)


Saved: results/submission/submission_debert.csv
